In [30]:
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.svm import LinearSVC
import numpy as np
from linear_models import LinearOneVsAllClassifier, LinearBinaryClassifier
import os
from utils import generate_exp_data, subset_binary_data, find_noise_bounds_binary

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
mnist_train_images = np.copy(mnist.train.images)
mnist_train_labels = np.argmax(mnist.train.labels, axis=1)
mnist_test_images = mnist.test.images
mnist_test_labels = np.argmax(mnist.test.labels, axis=1)

In [ ]:
num_models = 5
mnist_num_dim = 28 * 28
linear_models = []
zeroed_features_list = []
for i in xrange(num_models):
    sparse_mnist_images = np.copy(mnist_images)
    zeroed_features = np.random.randint(0, mnist_num_dim, int(.75 * mnist_num_dim))
    zeroed_features_list.append(zeroed_features)
    sparse_mnist_images[:, zeroed_features] = 0.0
    model = LinearSVC(loss='hinge')
    model.fit(sparse_mnist_images, mnist_labels)
    model = LinearOneVsAllClassifier(10, model.coef_, model.intercept_)
    linear_models.append(model)

In [ ]:
[model.evaluate(mnist_test_images, mnist_test_labels) for model in linear_models]

In [ ]:
# check that the models have no nonzero weights in any of the zeroed out dimensions
[sum(linear_models[i].weights[:,zeroed_features_list[i]].reshape(-1,)) for i in xrange(len(linear_models))]

In [ ]:
os.mkdir('linear_models')

In [ ]:
for i, model in enumerate(linear_models):
    np.save('linear_models/w_{}.npy'.format(i), model.weights)
    np.save('linear_models/b_{}.npy'.format(i), model.bias)

In [ ]:
X_exp, Y_exp = generate_exp_data(1000, mnist_test_images, mnist_test_labels, linear_models)

In [ ]:
[model.evaluate(X_exp, Y_exp) for model in linear_models]

In [ ]:
os.mkdir('linear_experiments_data')

In [ ]:
np.save('linear_experiments_data/X_exp_multi.npy', X_exp)
np.save('linear_experiments_data/Y_exp_multi.npy', Y_exp)

# Setup Binary Experiments

In [4]:
binary_train_images, binary_train_labels = subset_binary_data(mnist_train_images, mnist_train_labels, {4: -1, 9:1})
binary_test_images, binary_test_labels = subset_binary_data(mnist_test_images, mnist_test_labels, {4: -1, 9:1})

In [9]:
binary_test_images.shape

(1991, 784)

In [14]:
binary_models = []
num_models = 5
mnist_num_dim = 28 * 28
linear_models = []
zeroed_features_list = []
for i in xrange(num_models):
    sparse_binary_images = np.copy(binary_train_images)
    zeroed_features = np.random.randint(0, mnist_num_dim, int(.75 * mnist_num_dim))
    zeroed_features_list.append(zeroed_features)
    sparse_binary_images[:, zeroed_features] = 0.0
    model = LinearSVC(loss='hinge')    
    model.fit(sparse_binary_images, binary_train_labels)
    model = LinearBinaryClassifier(model.coef_.T, model.intercept_)
    binary_models.append(model)

In [15]:
[model.evaluate(binary_test_images, binary_test_labels) for model in binary_models]

[0.9552988448016072,
 0.9688598694123556,
 0.9648417880462079,
 0.9673530889000502,
 0.9653440482169764]

In [18]:
[sum(binary_models[i].weights[zeroed_features_list[i]].reshape(-1,)) for i in xrange(len(binary_models))]

[0.0, 0.0, 0.0, 0.0, 0.0]

In [16]:
os.mkdir('linear_models/binary')

In [19]:
for i, model in enumerate(binary_models):
    np.save('linear_models/binary/w_{}.npy'.format(i), model.weights)
    np.save('linear_models/binary/b_{}.npy'.format(i), model.bias)

In [25]:
X_exp_binary, Y_exp_binary = generate_exp_data(1000, binary_test_images, binary_test_labels, binary_models)

In [26]:
[model.evaluate(X_exp_binary, Y_exp_binary) for model in binary_models]

[1.0, 1.0, 1.0, 1.0, 1.0]

In [28]:
print X_exp_binary.shape
print Y_exp_binary.shape

(1000, 784)
(1000,)


In [29]:
os.mkdir('linear_experiments_data/binary')
np.save('linear_experiments_data/binary/X_exp.npy', X_exp_binary)
np.save('linear_experiments_data/binary/Y_exp.npy', Y_exp_binary)

In [32]:
min_bounds, max_bounds = find_noise_bounds_binary(binary_models, X_exp_binary, Y_exp_binary)

In [33]:
print np.median(min_bounds), np.median(max_bounds)

0.28278552220426245 0.4373595144920036
